In [1]:
import numpy as np
import cplex
import random

In [ ]:
def get_job_to_position(variable_names):
    job_to_position_onlyone = []
    for i in range((n_stage - 1) * n_job):
        job_to_position_onlyone.append([variable_names, [0] * i * n_job + [1] * n_job + [0] * ((n_stage - 1) * n_job * n_job - (i + 1) * n_job)])
    return job_to_position_onlyone

In [ ]:
def get_position_to_job(variable_names):
    position_to_job_onlyone = []
    for i in range((n_stage - 1) * n_job):
        coeff = np.zeros(((n_stage) * n_job, n_job))
        if i < n_job:
            coeff[0:n_job, i] = 1
        else:
            coeff[n_job:, i - n_job] = 1
        coeff = coeff.reshape((1, coeff.size))
        position_to_job_onlyone.append([variable_names, coeff.tolist()[0]])
    return position_to_job_onlyone

In [33]:
def solve_coeff_deri_F6_x(variable_names):
    tval=[random.randint(0,1) for i in range((n_stage - 1) * n_job)]
    print(tval)
    lamd=[1 for i in range((n_stage - 1) * n_job * n_job * n_job)]
    xval=[random.randint(0,1) for i in range((n_stage - 1) * n_job * n_job)]
    print(xval[0:6])
    print(xval[6:12])
    print(xval[12:18])
    print(xval[18:24])
    print(xval[24:30])
    print(xval[30:36])
    print(xval[36:42])
    print(xval[42:48])
    ProcessTime=[random.randint(0,1) for i in range((n_stage - 1) * n_job)]
    print(ProcessTime)
    x_coeff=[0 for i in range((n_stage - 1) * n_job * n_job)]
    for i in range(n_job):
        for j in range(n_stage-1):
            t1 = i + j * n_job
            for r in range(n_batch * pcast):
                for k in range(n_machine_perstage):
                    t2 = i * n_job + j * n_job * n_job + k * n_batch * pcast + r
                    for i1 in range(n_job):
                        if r == n_batch * pcast - 1:
                            t3 = i1 * n_job + i * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r - 1
                            t4 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r - 1 
                            t5 = i1 + j * n_job
                            x_coeff[t2] += lamd[t3] * ((0.5 * ((xval[t2] + xval[t4])**2 - xval[t2] - xval[t4]) + tval[t1] -  tval[t5]) * (xval[t2] + xval[t4] - 0.5) - 0.5 * (2 * ProcessTime[t5] + 1) * (xval[t4] - xval[t2]))           
                        elif r == 0:
                            t3 = i * n_job + i1 * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r
                            t4 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r + 1
                            t5 = i1 + j * n_job
                            x_coeff[t2] += lamd[t3] * ((0.5 * ((xval[t2] + xval[t4])**2 - xval[t2] - xval[t4]) + tval[t5] -  tval[t1]) * (xval[t2] + xval[t4] - 0.5) + 0.5 * (2 * ProcessTime[t1] + 1) * (xval[t2] - xval[t4]))           
                        else:
                            t3 = i * n_job + i1 * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r
                            t4 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r + 1
                            t5 = i1 + j * n_job
                            t6 = i1 * n_job + i * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r - 1
                            t7 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r - 1
                            x_coeff[t2] += lamd[t3] * ((0.5 * ((xval[t2] + xval[t4])**2 - xval[t2] - xval[t4]) + tval[t5] -  tval[t1]) * (xval[t2] + xval[t4] - 0.5) + 0.5 * (2 * ProcessTime[t1] + 1) * (xval[t2] - xval[t4]))
                            x_coeff[t2] += lamd[t6] * ((0.5 * ((xval[t2] + xval[t7])**2 - xval[t2] - xval[t7]) + tval[t1] -  tval[t5]) * (xval[t2] + xval[t7] - 0.5) - 0.5 * (2 * ProcessTime[t5] + 1) * (xval[t7] - xval[t2]))           
    return x_coeff

In [34]:
n_dim = (n_stage - 1) * n_job * n_job 
variable_names = ['x'+ str(i) for i in range(n_dim)]
x_coeff = solve_coeff_deri_F6_x(variable_names)
print(x_coeff)

[0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1]
[1, 0, 1, 0, 0, 0]
[1, 0, 0, 0, 1, 1]
[1, 0, 1, 0, 1, 1]
[0, 0, 0, 1, 0, 0]
[1, 0, 1, 0, 1, 1]
[1, 1, 1, 1, 0, 0]
[1, 0, 1, 1, 1, 1]
[1, 0, 1, 1, 0, 1]
[0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1]
[7.0, -6.5, 2.0, -0.5, -2.5, -2.5, 3.0, -5.5, -2.5, -0.5, 12.0, 9.0, 3.0, -5.5, 6.0, -0.5, 12.0, 9.0, -2.5, -10.5, -0.5, 14.0, -5.5, -2.5, 3.0, -5.5, 6.0, -0.5, 12.0, 9.0, 8.0, 18.0, 6.0, 8.0, -6.5, -2.5, 12.0, -11.5, 6.0, 11.0, 17.0, 6.0, 12.0, -11.5, 6.0, 11.0, -14.0, 6.0, -0.5, 13.0, -3.5, -1.5, -10.0, 12.0, -0.5, 12.0, -3.5, -1.5, 17.0, 6.0, 12.0, -11.5, 6.0, 11.0, -14.0, 6.0, 6.0, 16.0, -3.5, 5.0, 15.0, 12.0]


In [47]:
def solve_coeff_F4_x(variable_names):
    lamd=[1 for i in range((n_stage - 1) * n_job * n_job * n_job)]
    ProcessTime=[random.randint(30,36) for i in range((n_stage - 1) * n_job)]
    x_coeff=[0 for i in range((n_stage - 1) * n_job * n_job)]
    for i in range(n_job):
        for j in range(n_stage-1):
            t1 = i + j * n_job
            for r in range(n_batch * pcast):
                for k in range(n_machine_perstage):
                    t2 = i * n_job + j * n_job * n_job + k * n_batch * pcast + r
                    for i1 in range(n_job):
                        if r == n_batch * pcast - 1:
                            t3 = i * n_job + i1 * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + n_batch * pcast - 1
                            x_coeff[t2] += 0.25 * (2 * ProcessTime[t1] + 1) * lamd[t3]
                        elif r == 0:
                            t3 = i * n_job + i1 * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast
                            x_coeff[t2] += 0.25 * (2 * ProcessTime[t1] + 1) * lamd[t3]
                        else:
                            t3 = i * n_job + i1 * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r
                            t4 = i1 * n_job + i * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r + 1
                            x_coeff[t2] += 0.25 * (2 * ProcessTime[t1] + 1) * (lamd[t3] + lamd[t4])
    return x_coeff
                        
    
         

In [53]:
n_dim = (n_stage - 1) * n_job * n_job 
variable_names = ['x'+ str(i) for i in range(n_dim)]
x_coeff = solve_coeff_F4_x(variable_names)
print(x_coeff)

[100.5, 201.0, 100.5, 100.5, 201.0, 100.5, 91.5, 183.0, 91.5, 91.5, 183.0, 91.5, 97.5, 195.0, 97.5, 97.5, 195.0, 97.5, 100.5, 201.0, 100.5, 100.5, 201.0, 100.5, 100.5, 201.0, 100.5, 100.5, 201.0, 100.5, 94.5, 189.0, 94.5, 94.5, 189.0, 94.5, 109.5, 219.0, 109.5, 109.5, 219.0, 109.5, 97.5, 195.0, 97.5, 97.5, 195.0, 97.5, 97.5, 195.0, 97.5, 97.5, 195.0, 97.5, 97.5, 195.0, 97.5, 97.5, 195.0, 97.5, 103.5, 207.0, 103.5, 103.5, 207.0, 103.5, 100.5, 201.0, 100.5, 100.5, 201.0, 100.5]


In [45]:
def solve_coeff_deri_F6_t(variable_names):
    tval=[random.randint(0,1) for i in range((n_stage - 1) * n_job)]
    print(tval)
    lamd=[1 for i in range((n_stage - 1) * n_job * n_job * n_job)]
    xval=[random.randint(0,1) for i in range((n_stage - 1) * n_job * n_job)]
    print(xval[0:6])
    print(xval[6:12])
    print(xval[12:18])
    print(xval[18:24])
    print(xval[24:30])
    print(xval[30:36])
    print(xval[36:42])
    print(xval[42:48])
    print(xval[48:54])
    print(xval[54:60])
    print(xval[60:66])
    print(xval[66:72])
    t_coeff=[0 for i in range((n_stage - 1) * n_job)]
    for i in range(n_job):
        for j in range(n_stage-1):
            t1 = i + j * n_job
            for r in range(n_batch * pcast):
                for k in range(n_machine_perstage):
                    t2 = i * n_job + j * n_job * n_job + k * n_batch * pcast + r
                    for i1 in range(n_job):
                        if r == n_batch * pcast - 1:
                            t3 = i1 * n_job + i * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r - 1
                            t4 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r - 1 
                            t5 = i1 + j * n_job
                            t_coeff[t1] += lamd[t3] * (0.5 * ((xval[t2] + xval[t4])**2 - xval[t2] - xval[t4]) + tval[t1] -  tval[t5])          
                        elif r == 0:
                            t3 = i * n_job + i1 * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r
                            t4 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r + 1
                            t5 = i1 + j * n_job
                            t_coeff[t1] -= lamd[t3] * (0.5 * ((xval[t2] + xval[t4])**2 - xval[t2] - xval[t4]) + tval[t5] -  tval[t1])
                        else:
                            t3 = i * n_job + i1 * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r
                            t4 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r + 1
                            t5 = i1 + j * n_job
                            t6 = i1 * n_job + i * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r - 1
                            t7 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r - 1
                            t_coeff[t1] -= lamd[t3] * (0.5 * ((xval[t2] + xval[t4])**2 - xval[t2] - xval[t4]) + tval[t5] -  tval[t1]) 
                            t_coeff[t1] += lamd[t6] * (0.5 * ((xval[t2] + xval[t7])**2 - xval[t2] - xval[t7]) + tval[t1] -  tval[t5])            
    return t_coeff

In [49]:
n_dim = (n_stage - 1) * n_job 
variable_names = ['t'+ str(i) for i in range(n_dim)]
t_coeff = solve_coeff_deri_F6_t(variable_names)
print(t_coeff)

[1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1]
[0, 1, 1, 0, 1, 1]
[0, 1, 1, 1, 0, 1]
[0, 1, 1, 1, 1, 1]
[1, 0, 0, 1, 1, 1]
[1, 1, 0, 1, 1, 1]
[1, 0, 0, 0, 1, 1]
[0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 1, 1]
[0, 0, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 0]
[0, 0, 1, 1, 0, 0]
[1, 1, 0, 1, 1, 1]
[39.0, -12.0, -14.0, 26.0, -22.0, -17.0, -22.0, 29.0, 27.0, -29.0, -26.0, 21.0]


In [84]:
def solve_coeff_quad_t():
    lamd=[1 for i in range((n_stage - 1) * n_job * n_job * n_job)]
    quad_t_coeff=np.zeros(((n_stage - 1) * n_job, (n_stage - 1) * n_job))
    for i in range(n_job):
        for j in range(n_stage-1):
            t3 = i + j * n_job
            for i1 in range(n_job):
                t4 = i1 + j * n_job 
                for r in range(n_batch * pcast):
                    for k in range(n_machine_perstage):
                        t1 = i * n_job + i1 * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r
                        t2 = i1 * n_job + i * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r
                        print(t3, t4)
                        if t3 == t4:
                            quad_t_coeff[t3,t4] += 0.5 * (lamd[t1] + lamd[t2])
                        else:
                            quad_t_coeff[t3,t4] -= 0.5 * (lamd[t1] + lamd[t2])
    return quad_t_coeff
            
    

In [85]:
quad_t_coeff = solve_coeff_quad_t()
print(quad_t_coeff)

0 0
0 0
0 0
0 0
0 0
0 0
0 1
0 1
0 1
0 1
0 1
0 1
0 2
0 2
0 2
0 2
0 2
0 2
0 3
0 3
0 3
0 3
0 3
0 3
0 4
0 4
0 4
0 4
0 4
0 4
0 5
0 5
0 5
0 5
0 5
0 5
6 6
6 6
6 6
6 6
6 6
6 6
6 7
6 7
6 7
6 7
6 7
6 7
6 8
6 8
6 8
6 8
6 8
6 8
6 9
6 9
6 9
6 9
6 9
6 9
6 10
6 10
6 10
6 10
6 10
6 10
6 11
6 11
6 11
6 11
6 11
6 11
1 0
1 0
1 0
1 0
1 0
1 0
1 1
1 1
1 1
1 1
1 1
1 1
1 2
1 2
1 2
1 2
1 2
1 2
1 3
1 3
1 3
1 3
1 3
1 3
1 4
1 4
1 4
1 4
1 4
1 4
1 5
1 5
1 5
1 5
1 5
1 5
7 6
7 6
7 6
7 6
7 6
7 6
7 7
7 7
7 7
7 7
7 7
7 7
7 8
7 8
7 8
7 8
7 8
7 8
7 9
7 9
7 9
7 9
7 9
7 9
7 10
7 10
7 10
7 10
7 10
7 10
7 11
7 11
7 11
7 11
7 11
7 11
2 0
2 0
2 0
2 0
2 0
2 0
2 1
2 1
2 1
2 1
2 1
2 1
2 2
2 2
2 2
2 2
2 2
2 2
2 3
2 3
2 3
2 3
2 3
2 3
2 4
2 4
2 4
2 4
2 4
2 4
2 5
2 5
2 5
2 5
2 5
2 5
8 6
8 6
8 6
8 6
8 6
8 6
8 7
8 7
8 7
8 7
8 7
8 7
8 8
8 8
8 8
8 8
8 8
8 8
8 9
8 9
8 9
8 9
8 9
8 9
8 10
8 10
8 10
8 10
8 10
8 10
8 11
8 11
8 11
8 11
8 11
8 11
3 0
3 0
3 0
3 0
3 0
3 0
3 1
3 1
3 1
3 1
3 1
3 1
3 2
3 2
3 2
3 2
3 2
3 2
3 3
3 3
3 3
3 3
3 3
3 3
3 4


In [ ]:
def solve_LP_TU():
    Lagrangian_LP_TU = cplex.Cplex()
    Lagrangian_LP_TU.objective.set_sense(Lagrangian_LP_TU.objective.sense.maximize)
    n_dim = (n_stage - 1) * n_job * n_job
    LP_TU_obj = [random.random() for i in range(n_dim)]
    variable_names = ['x'+ str(i) for i in range(n_dim)]
    ub = [1] * n_dim
    
    Lagrangian_LP_TU.variables.add(obj = LP_TU_obj, ub = ub, names = variable_names)    
    
    constraints_rows = get_job_to_position(variable_names) + get_position_to_job(variable_names)
    n_constraints = len(constraints_rows)
    senses = 'E'* n_constraints
    rhs = [1] * n_constraints
    row_names = ['c'+ str(i) for i in range(n_constraints)]
    print(n_dim, len(constraints_rows), len(senses), len(rhs), len(row_names))
    Lagrangian_LP_TU.linear_constraints.add(lin_expr = constraints_rows, senses = senses, rhs = rhs, names = row_names)
    Lagrangian_LP_TU.solve()
    Lagrangian_LP_TU.write('Lagrangian_LP_TU.lp')
    return Lagrangian_LP_TU
    

In [ ]:
def print_LP_result(my_prob):
    x_solution = []
    for i, x in enumerate(my_prob.solution.get_values()):
        x_solution.append(x)
    x_solution = np.array(x_solution)
    print(x_solution.shape)
    x_solution = x_solution.reshape(((n_stage-1)* n_job, n_job))
    x_solution = {'stage1': x_solution[0:n_job, :], 'stage2': x_solution[n_job:, :]}
    print('stage 1')
    print(x_solution['stage1'])
    print('stage 2')
    print(x_solution['stage2'])
    print ("Solution value:   ",  my_prob.solution.get_objective_value())

In [63]:
n_stage, pcast, n_batch, n_machine_perstage = 3, 1, 3, 2
stage_name = ['steelmaking', 'refining', 'casting']
n_machine = dict(zip(stage_name, [n_machine_perstage for i in range(n_stage)]))
n_job = pcast * n_batch * n_machine['steelmaking']
assert(len(stage_name) == n_stage)

print("Number of stage:", n_stage)
print("Cast number of per machine:", pcast)
print("Number of job per cast:", n_batch)
print('Number of machine:', n_machine)
print('Number of job:', n_job)

n_position = dict(zip(stage_name[0:n_stage - 1], [n_job / n_machine['steelmaking'] ] * (n_stage - 1)))
print('n_position = ', n_position)
assert(len(n_machine) == n_stage)
assert(len(n_position) == n_stage - 1)

Number of stage: 3
Cast number of per machine: 1
Number of job per cast: 3
Number of machine: {'steelmaking': 2, 'refining': 2, 'casting': 2}
Number of job: 6
n_position =  {'steelmaking': 3.0, 'refining': 3.0}


In [ ]:
Lagrangian_LP_TU = solve_LP_TU()
print_LP_result(Lagrangian_LP_TU)

In [ ]:
print_LP_result(Lagrangian_LP_TU)